# Tasca 5: Exploració de dades


Roger Gallart - 24 abril 2021

## Nivell 1



### Exercici 1
*Descarrega el data set [Airlines Delay: Airline on-time statistics and delay causes](https://www.kaggle.com/giovamata/airlinedelaycauses) i carrega’l a un pandas Dataframe. Explora les dades que conté, i queda’t únicament amb les columnes que consideris rellevants.*

In [66]:
import pandas as pd
import numpy as np

In [67]:
# He pujat el dataset a una carpeta de Google Drive
# La unitat de GoogleDrive està muntada a la carpeta '/gdrive/Mydrive' de Colab

# Llegim el fitxer CSV
# La primera columna del fitxer sembla ser algún tipus de ID. L'utilitzem com a index del dataframe
df = pd.read_csv('/content/drive/MyDrive/ITAcademy/DelayedFlights.csv', index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


La descripció de les variables està disponible en aquesta web: http://stat-computing.org/dataexpo/2009/the-data.html


In [68]:
# Visualitzem les darreres 10 files, per exemple
df.tail(10)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
7009699,2008,12,13,6,1254.0,1221,1420.0,1359,DL,1609,N977DL,86.0,98.0,64.0,21.0,33.0,IND,ATL,432,9.0,13.0,0,N,0,0.0,0.0,0.0,0.0,21.0
7009701,2008,12,13,6,1528.0,1500,1720.0,1642,DL,1611,N395DN,112.0,102.0,79.0,38.0,28.0,SLC,PHX,507,4.0,29.0,0,N,0,16.0,0.0,10.0,0.0,12.0
7009702,2008,12,13,6,1531.0,1522,1822.0,1823,DL,1612,N916DN,111.0,121.0,88.0,-1.0,9.0,MCI,ATL,692,9.0,14.0,0,N,0,NaN,NaN,NaN,NaN,NaN
7009705,2008,12,13,6,921.0,830,1112.0,1008,DL,1616,N907DE,111.0,98.0,82.0,64.0,51.0,ATL,PBI,545,8.0,21.0,0,N,0,51.0,0.0,13.0,0.0,0.0
7009709,2008,12,13,6,1552.0,1520,1735.0,1718,DL,1620,N905DE,43.0,58.0,27.0,17.0,32.0,HSV,ATL,151,9.0,7.0,0,N,0,0.0,0.0,0.0,0.0,17.0
7009710,2008,12,13,6,1250.0,1220,1617.0,1552,DL,1621,N938DL,147.0,152.0,120.0,25.0,30.0,MSP,ATL,906,9.0,18.0,0,N,0,3.0,0.0,0.0,0.0,22.0
7009717,2008,12,13,6,657.0,600,904.0,749,DL,1631,N3743H,127.0,109.0,78.0,75.0,57.0,RIC,ATL,481,15.0,34.0,0,N,0,0.0,57.0,18.0,0.0,0.0
7009718,2008,12,13,6,1007.0,847,1149.0,1010,DL,1631,N909DA,162.0,143.0,122.0,99.0,80.0,ATL,IAH,689,8.0,32.0,0,N,0,1.0,0.0,19.0,0.0,79.0
7009726,2008,12,13,6,1251.0,1240,1446.0,1437,DL,1639,N646DL,115.0,117.0,89.0,9.0,11.0,IAD,ATL,533,13.0,13.0,0,N,0,NaN,NaN,NaN,NaN,NaN
7009727,2008,12,13,6,1110.0,1103,1413.0,1418,DL,1641,N908DL,123.0,135.0,104.0,-5.0,7.0,SAT,ATL,874,8.0,11.0,0,N,0,NaN,NaN,NaN,NaN,NaN


In [69]:
# Revisem les caracteristiques de les dades importades
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1936758 entries, 0 to 7009727
Data columns (total 29 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Year               int64  
 1   Month              int64  
 2   DayofMonth         int64  
 3   DayOfWeek          int64  
 4   DepTime            float64
 5   CRSDepTime         int64  
 6   ArrTime            float64
 7   CRSArrTime         int64  
 8   UniqueCarrier      object 
 9   FlightNum          int64  
 10  TailNum            object 
 11  ActualElapsedTime  float64
 12  CRSElapsedTime     float64
 13  AirTime            float64
 14  ArrDelay           float64
 15  DepDelay           float64
 16  Origin             object 
 17  Dest               object 
 18  Distance           int64  
 19  TaxiIn             float64
 20  TaxiOut            float64
 21  Cancelled          int64  
 22  CancellationCode   object 
 23  Diverted           int64  
 24  CarrierDelay       float64
 25  WeatherDelay      

Observacions:
  - 1.9M de files i 29 columnes
  - Veiem alguns problemes en el format d'alguns camps que no són deguts a la importació sino que ja estaven guardats de manera "inadecuada" al CSV d'origen(si ho mires amb un editor de text):
    - Els camps de temps com DepTime, CRSDepTime, etc haurien de tenir format (hhmm) segons la descripció però realment estan escrits al CSV com un float (1254.0 == 12h54min) o com un enter (830 == 0830 = 8h30min). 
    - El número de vol es va guardar com un número enlloc d'un string, i llavors els que començaven per "0" estan mal guardarts e.g. el vol 61 hauria de ser 0061.
    

In [70]:
# Pandas té una molt bona API per tractar dates i series temporals
# Creem una columna de tipus datetime utilitzant la data i la hora oficial del vol (CRSDepTime)

df['CRSDepTime'] = df['CRSDepTime'].astype(str).apply(lambda x: x.zfill(4)) # zfill() afegeix zeros a l'esquerra d'un string
df['DateTime'] = df[['Year','Month','DayofMonth']].astype(str).apply('-'.join,1)\
                + ' ' + df['CRSDepTime'].str[:2] \
                + ':' + df['CRSDepTime'].str[2:] + ':00'
df['DateTime'] = pd.to_datetime(df['DateTime']) 

In [71]:
# Llista dels noms de les columnes
df.columns

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay',
       'DateTime'],
      dtype='object')

In [72]:
# Eliminem les columnes que no ens interessen
cols_eliminades = ['Year', 'Month', 'DayofMonth', 'DayOfWeek','DepTime',
                  'CRSDepTime','ArrTime', 'CRSArrTime', 'TailNum']

df.drop(columns=cols_eliminades, inplace=True)

In [73]:
# Reordenem les columnes
df = df[['DateTime','UniqueCarrier', 'FlightNum', 'ActualElapsedTime', 'CRSElapsedTime','AirTime', 'ArrDelay', 'DepDelay',
       'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut', 'Cancelled',
       'CancellationCode', 'Diverted', 'CarrierDelay', 'WeatherDelay',
       'NASDelay', 'SecurityDelay', 'LateAircraftDelay', ]]

In [74]:
# Convertim FlightNum a un string de 4 caràcters (e.g. 23 --> '0023')
df.FlightNum = df.FlightNum.astype(str).apply(lambda x: x.zfill(4))

# Convertim Cancelled i Diverted a tipus boolean
df.Cancelled = df.Cancelled.astype(bool)
df.Diverted = df.Diverted.astype(bool)

# Convertim CancellationCode a tipus category
df.CancellationCode = df.CancellationCode.astype('category')


In [75]:
df.head()

,DateTime,UniqueCarrier,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008-01-03 19:55:00,WN,0335,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,False,N,False,NaN,NaN,NaN,NaN,NaN
1,2008-01-03 07:35:00,WN,3231,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,False,N,False,NaN,NaN,NaN,NaN,NaN
2,2008-01-03 06:20:00,WN,0448,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,False,N,False,NaN,NaN,NaN,NaN,NaN
4,2008-01-03 17:55:00,WN,3920,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,False,N,False,2.0,0.0,0.0,0.0,32.0
5,2008-01-03 19:15:00,WN,0378,101.0,115.0,87.0,11.0,25.0,IND,JAX,688,4.0,10.0,False,N,False,NaN,NaN,NaN,NaN,NaN


### Exercici 2
*Fes un informe complet del data set:*

- *Resumeix estadísticament les columnes d’interès*
- *Troba quantes dades faltants hi ha per columna*
- *Crea columnes noves (velocitat mitjana del vol, si ha arribat tard o no...)*
- *Taula de les aerolínies amb més endarreriments acumulats*
- *Quins són els vols més llargs? I els més endarrerits?*
- *Etc.*

#### Resum estadístic

In [76]:
# Separem variables numèriques i categòriques, per més claredat
col_categoriques = [ 'DateTime', 'UniqueCarrier', 'FlightNum','Origin',
                    'Dest','Cancelled', 'CancellationCode', 'Diverted'] #Incluïm DateTime com a categòrica

col_numeriques = [x for x in df.columns if x not in col_categoriques] # La resta de columnes que no son categòriques

In [77]:
# Resum estadístic de variables numèriques
df[col_numeriques].describe().applymap("{0:.2f}".format)

,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Distance,TaxiIn,TaxiOut,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,1928371.00,1936560.00,1928371.00,1928371.00,1936758.00,1936758.00,1929648.00,1936303.00,1247488.00,1247488.00,1247488.00,1247488.00,1247488.00
mean,133.31,134.30,108.28,42.20,43.19,765.69,6.81,18.23,19.18,3.70,15.02,0.09,25.30
std,72.06,71.34,68.64,56.78,53.40,574.48,5.27,14.34,43.55,21.49,33.83,2.02,42.05
min,14.00,-25.00,0.00,-109.00,6.00,11.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,80.00,82.00,58.00,9.00,12.00,338.00,4.00,10.00,0.00,0.00,0.00,0.00,0.00
50%,116.00,116.00,90.00,24.00,24.00,606.00,6.00,14.00,2.00,0.00,2.00,0.00,8.00
75%,165.00,165.00,137.00,56.00,53.00,998.00,8.00,21.00,21.00,0.00,15.00,0.00,33.00
max,1114.00,660.00,1091.00,2461.00,2467.00,4962.00,240.00,422.00,2436.00,1352.00,1357.00,392.00,1316.00


In [78]:
# Resum estadístic de variables categòriques i de la variable DateTime
df[col_categoriques].describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  


,DateTime,UniqueCarrier,FlightNum,Origin,Dest,Cancelled,CancellationCode,Diverted
count,1936758,1936758,1936758,1936758,1936758,1936758,1936758,1936758
unique,271193,20,7499,303,304,2,4,2
top,2008-12-22 06:00:00,WN,0016,ATL,ORD,False,N,False
freq,106,377602,1586,131613,108984,1936125,1936125,1929004
first,2008-01-01 00:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2008-12-31 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Dades que falten (missing values)

In [79]:
# Quantes dades falten per cada columna (en valor absolut i percentatge)
missing_values = df.isna().sum()
print("Nombre de dades que falten (en valor absolut):\n", missing_values)
print("\nNombre de dades que falten (en percentatge):\n", (missing_values*100/df.shape[0]))

Nombre de dades que falten (en valor absolut):
 DateTime                  0
UniqueCarrier             0
FlightNum                 0
ActualElapsedTime      8387
CRSElapsedTime          198
AirTime                8387
ArrDelay               8387
DepDelay                  0
Origin                    0
Dest                      0
Distance                  0
TaxiIn                 7110
TaxiOut                 455
Cancelled                 0
CancellationCode          0
Diverted                  0
CarrierDelay         689270
WeatherDelay         689270
NASDelay             689270
SecurityDelay        689270
LateAircraftDelay    689270
dtype: int64

Nombre de dades que falten (en percentatge):
 DateTime              0.000000
UniqueCarrier         0.000000
FlightNum             0.000000
ActualElapsedTime     0.433043
CRSElapsedTime        0.010223
AirTime               0.433043
ArrDelay              0.433043
DepDelay              0.000000
Origin                0.000000
Dest                  0.0

Veiem que bàsicament on falten més dades (35%) és a les columnes referents als motius dels delays (CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay).


#### Creació de noves columnes

##### Velocitat mitjana (AvgSpeed)


In [80]:
# Velocitat mitjana del vol (en knots = nusos = milles/hora)
# Distance [miles], AirTime [min]
df['AvgSpeed'] = 60*df.Distance/df.AirTime

##### Vol arribat amb retard (ArrIsDelayed)

In [81]:
# Si el vol ha arribat amb retard
# Nota: La Federal Aviation Administration (FAA) considera que és un retard quan 
# han passat més de 15min des de l'hora prevista

df['ArrIsDelayed'] = df['ArrDelay'] >= 15.0

##### Proporció de temps que l'avió està volant respecte de la durada total del vol

In [82]:
# Proporció de temps que l'avió està a l'aire respecte
# de la durada total del vol

df['AirTimeProportion'] = df['AirTime']/df['ActualElapsedTime']

##### Diferència entre durada real del vol (ActualElapsedTime) i la durada prevista (CRSElapsedTime)

In [83]:
# Diferència entre la durada real del vol i la durada prevista, en minuts
df['ElapsedTimeError'] = df['ActualElapsedTime']-df['CRSElapsedTime']

In [84]:
df.head(5)

,DateTime,UniqueCarrier,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,AvgSpeed,ArrIsDelayed,AirTimeProportion,ElapsedTimeError
0,2008-01-03 19:55:00,WN,0335,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,False,N,False,NaN,NaN,NaN,NaN,NaN,418.965517,False,0.906250,-22.0
1,2008-01-03 07:35:00,WN,3231,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,False,N,False,NaN,NaN,NaN,NaN,NaN,430.088496,False,0.882812,-17.0
2,2008-01-03 06:20:00,WN,0448,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,False,N,False,NaN,NaN,NaN,NaN,NaN,406.578947,False,0.791667,6.0
4,2008-01-03 17:55:00,WN,3920,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,False,N,False,2.0,0.0,0.0,0.0,32.0,401.298701,True,0.855556,0.0
5,2008-01-03 19:15:00,WN,0378,101.0,115.0,87.0,11.0,25.0,IND,JAX,688,4.0,10.0,False,N,False,NaN,NaN,NaN,NaN,NaN,474.482759,False,0.861386,-14.0


#### Taula de les aerolínies amb més endarreriments acumulats

Nota: només hem considerat endarreriments a l'arribada

In [85]:
df.groupby(by="UniqueCarrier").agg({'ArrIsDelayed' : np.sum})\
                              .sort_values(by='ArrIsDelayed', ascending=False)\
                              .rename(columns={"ArrIsDelayed":"Number of delays"})

,Number of delays
UniqueCarrier,
WN,203559
AA,132257
MQ,97555
UA,95465
OO,88991
DL,72252
XE,72008
US,59508
CO,58958


#### Quins són els vols més llargs?


**IMPORTANT: el número de vol no és unic, ni tan sols per a una mateixa aerolinia.**

Per exemple el vol B6-0001 un dia correspon a un vol de BUF a JFK i un altre dia de JFK a FLL. Per tant no té sentit agrupar per número de vol (FlightNum).

In [86]:
# Exemple de 2 vols amb el mateix número de vol (FlightNum)
df.loc[[6955227,6955375],:]

,DateTime,UniqueCarrier,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,AvgSpeed,ArrIsDelayed,AirTimeProportion,ElapsedTimeError
6955227,2008-12-02 08:25:00,B6,0001,87.0,80.0,57.0,25.0,18.0,BUF,JFK,301,7.0,23.0,False,N,False,18.0,0.0,7.0,0.0,0.0,316.842105,True,0.655172,7.0
6955375,2008-12-02 10:35:00,B6,0001,217.0,185.0,179.0,72.0,40.0,JFK,FLL,1069,22.0,16.0,False,N,False,25.0,0.0,32.0,0.0,15.0,358.324022,True,0.824885,32.0


In [87]:
# Els 5 vols més llargs
df.sort_values(by='ActualElapsedTime',ascending=False)\
    .head(5)[['DateTime','UniqueCarrier','FlightNum','Origin','Dest','ActualElapsedTime']]

,DateTime,UniqueCarrier,FlightNum,Origin,Dest,ActualElapsedTime
5180146,2008-09-09 09:20:00,HA,0021,SEA,HNL,1114.0
6980183,2008-12-19 13:35:00,CO,0015,EWR,HNL,790.0
3922427,2008-07-23 16:45:00,DL,0151,JFK,SFO,776.0
4811552,2008-08-02 13:35:00,CO,0015,EWR,HNL,750.0
4614554,2008-08-19 12:30:00,HA,0028,HNL,SEA,750.0


In [88]:
# Si agrupem per vols iguals (mateixa aerolinia i trajecte) i calculem la mitjana de durada real del vol
df.groupby(['UniqueCarrier','Origin','Dest']).agg({'ActualElapsedTime':np.mean})\
          .sort_values(by='ActualElapsedTime',ascending=False)\
          .rename(columns={'ActualElapsedTime':'Durada mitjana del vol (min)'})\
          .head(5)

,,,Durada mitjana del vol (min)
UniqueCarrier,Origin,Dest,
CO,EWR,HNL,637.958491
DL,ATL,HNL,583.228758
CO,HNL,EWR,575.175439
AA,ORD,HNL,551.869159
UA,ORD,HNL,536.202020


#### Quins són els més endarrerits ?

Nota: només considerem endarreriment a l'arribada

In [89]:
# Els 5 vols més endarrerits (a l'arribada)
df.sort_values(by='ArrDelay',ascending=False)\
    .head(5)[['DateTime','UniqueCarrier','FlightNum','Origin','Dest','ArrDelay']]

,DateTime,UniqueCarrier,FlightNum,Origin,Dest,ArrDelay
1018798,2008-02-03 18:20:00,NW,0808,HNL,MSP,2461.0
2235378,2008-04-10 14:17:00,NW,1699,CLT,MSP,2453.0
2832617,2008-05-06 15:30:00,NW,1107,RSW,DTW,1951.0
3387883,2008-06-20 17:10:00,MQ,3538,LIT,DFW,1707.0
6857047,2008-12-19 13:25:00,NW,0357,BOS,MSP,1655.0


In [90]:
# Si agrupem per vols iguals (mateixa aerolinia i trajecte) i calculem 
# l'endarreriment mitjà a l'arribada
df.groupby(['UniqueCarrier','Origin','Dest']).agg({'ArrDelay':np.mean})\
          .sort_values(by='ArrDelay',ascending=False)\
          .rename(columns={'ArrDelay':'Endarreriment mitjà (min)'})\
          .head(5)

,,,Endarreriment mitjà (min)
UniqueCarrier,Origin,Dest,
MQ,CMI,SPI,575.000000
US,DCA,IND,299.000000
YV,CID,DEN,284.000000
OH,STL,OMA,275.000000
NW,MSP,CID,274.666667


### Exercici 3
*Exporta el dataset net i amb les noves columnes a Excel.*

In [91]:
df.shape

(1936758, 25)

In [92]:
df.columns

Index(['DateTime', 'UniqueCarrier', 'FlightNum', 'ActualElapsedTime',
       'CRSElapsedTime', 'AirTime', 'ArrDelay', 'DepDelay', 'Origin', 'Dest',
       'Distance', 'TaxiIn', 'TaxiOut', 'Cancelled', 'CancellationCode',
       'Diverted', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay',
       'LateAircraftDelay', 'AvgSpeed', 'ArrIsDelayed', 'AirTimeProportion',
       'ElapsedTimeError'],
      dtype='object')

In [93]:
# Reordenem les columnes abans d'exportar les dades
df = df[['DateTime', 'UniqueCarrier', 'FlightNum', 'Origin', 'Dest', 'Distance',
         'AvgSpeed', 'ArrIsDelayed','ArrDelay', 'DepDelay', 'ActualElapsedTime',
         'CRSElapsedTime','ElapsedTimeError','AirTime','AirTimeProportion',
         'Diverted', 'CarrierDelay','WeatherDelay','NASDelay', 'SecurityDelay',
         'LateAircraftDelay', ]]

In [94]:
df.head()

,DateTime,UniqueCarrier,FlightNum,Origin,Dest,Distance,AvgSpeed,ArrIsDelayed,ArrDelay,DepDelay,ActualElapsedTime,CRSElapsedTime,ElapsedTimeError,AirTime,AirTimeProportion,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008-01-03 19:55:00,WN,0335,IAD,TPA,810,418.965517,False,-14.0,8.0,128.0,150.0,-22.0,116.0,0.906250,False,NaN,NaN,NaN,NaN,NaN
1,2008-01-03 07:35:00,WN,3231,IAD,TPA,810,430.088496,False,2.0,19.0,128.0,145.0,-17.0,113.0,0.882812,False,NaN,NaN,NaN,NaN,NaN
2,2008-01-03 06:20:00,WN,0448,IND,BWI,515,406.578947,False,14.0,8.0,96.0,90.0,6.0,76.0,0.791667,False,NaN,NaN,NaN,NaN,NaN
4,2008-01-03 17:55:00,WN,3920,IND,BWI,515,401.298701,True,34.0,34.0,90.0,90.0,0.0,77.0,0.855556,False,2.0,0.0,0.0,0.0,32.0
5,2008-01-03 19:15:00,WN,0378,IND,JAX,688,474.482759,False,11.0,25.0,101.0,115.0,-14.0,87.0,0.861386,False,NaN,NaN,NaN,NaN,NaN


El màxim que permet Excel són 1048576 files i 16384 columnes. Ho guardem en un fitxer CSV enlloc de un Excel


In [95]:
# Guardar en un CSV
path = '/content/drive/MyDrive/ITAcademy/clean_DelayedFlights.csv'
df.to_csv(path)

El fitxer CSV és massa gran per pujar-lo al GiHub (max 100MB). Està penjat al meu google Drive:

https://drive.google.com/file/d/1mM3H94DHEDg0q6eSb5N3Jy-l1kyzQvAS/view?usp=sharing